In [1]:
%%time
import os
import time
import sys 
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

Wall time: 1.17 s


# Example Notebook

## 1. Import the Package SmartControl

In [2]:
%%time
path = 'D:\\Repos\\PirnaCaseStudy'
sys.path.append(path) #indicate the the repository is
import SmartControl as sc

Wall time: 8.05 s


In [41]:
sys.getsizeof(43) 

28

## 2. Querying data using the module *queries*

In [3]:
database_fn = 'Data/Database.db'
os.chdir(path)

#Getting the database
Get = sc.queries.Get(database_fn) #instantiates the database 

In [43]:
import folium

In [44]:
Map = folium.Map(zoom_start = 1)

In [45]:
type(Map)

folium.folium.Map

### 2.1 Fetching database status

In [4]:

VariableID = Get.VariableID('h_level') # The ID of the variable
APIDate_t , APIDate_ts = Get.APIDate(23) # The last date that the monitoring id was updated


print(VariableID)
print(f'Pandas Timestamp: {APIDate_t}')
print(f'Unix Timestamp counted from 1970 onwards: {APIDate_ts}')

0
Pandas Timestamp: 2023-01-06 13:00:00
Unix Timestamp counted from 1970 onwards: 1673010000


### 2.2 Fetching important structure data

In [5]:
%%time

MonitoringPointData_df = Get.MonitoringPointData(GageData = 1)
GageData_df = Get.GageData

Wall time: 15.6 ms


If one wishes to visualize the data, just type it into a cell

In [7]:
DiverData_df

,MonitoringPointID,MonitoringPointName,DiverDepth,ReferenceAltitude,WellDepth
0,23,GWM02,12.3,117.98,14.0


#### Diver Status

It returns Dataframe joining divers, wells, drills and the current status with regards to the divers' last update.

In [13]:
%%time
DiverStatus_df = Get.DiverStatus() 

Wall time: 8.69 s


In [25]:
DiverStatus_df.shape

(20, 15)

In [23]:
DiverStatus_df [ (DiverStatus_df.IOT == 0) & (DiverStatus_df.Functioning == 1)].shape

(8, 15)

In [9]:
DiverStatus_df [DiverStatus_df.MonitoringPointName.str.startswith('GW') ]

,PointID,MonitoringPointName,ReferenceAltitude,Type,TypeOfAltitude,MonitoringPointID,DiverID,DiverName,IOT,Functioning,PointsID,E,N,NextUpdate_t,NextUpdate_ts
11,26,GWM02,117.98,Well,Top Case,23,11,I-2,1,1,26,13.923774,50.965905,2022-12-15 14:00:00,1671112800
12,27,GWM03,117.88,Well,Top Case,24,12,I-3,1,1,27,13.923764,50.965858,2022-12-15 14:00:00,1671112800
13,29,GWM05,117.87,Well,Top Case,26,13,I-5,1,1,29,13.923778,50.965812,2022-12-15 14:00:00,1671112800
14,30,GWM06,117.93,Well,Top Case,27,14,I-6,1,0,30,13.923829,50.965826,2022-01-17 09:00:00,1642410000


### 2.3 Historical data

Fetching the historical values

In [4]:
%%time
# Get.LongTimeSeries(0)
# Get.ShortTimeSeries(FilterVariableID = 0, FilterMonitoringPoint = 'G01')
Get.Isolines(2022, 12, 7, 2)
# Get.HydroProfile()

Wall time: 817 ms


In [6]:
status_df = Get.DiverStatus()

In [7]:
status_df

,PointID,MonitoringPointName,ReferenceAltitude,Type,TypeOfAltitude,MonitoringPointID,DiverID,DiverName,IOT,Functioning,PointsID,E,N,NextUpdate_t,NextUpdate_ts
0,0,G01,119.49,Well,Top Case,0,0,S01,0,1,0,13.922657,50.965827,2023-01-05 16:00:00,1672934400
1,3,G04,119.19,Well,Top Case,1,1,S02,0,0,3,13.923426,50.965848,2022-01-17 12:46:40,1642423600
2,6,G10,119.29,Well,Top Case,3,2,S03,0,1,6,13.923311,50.966071,2022-01-17 12:46:40,1642423600
3,7,G11,116.95,Well,Top Case,4,3,S04,0,1,7,13.923868,50.965732,2022-01-17 12:46:40,1642423600
4,8,G12,116.57,Well,Top Case,5,4,S05,0,1,8,13.923290,50.965675,2022-01-17 12:46:40,1642423600
5,9,G13,116.97,Well,Top Case,6,5,S06,0,1,9,13.922816,50.965641,2022-01-17 12:46:40,1642423600
6,10,G15,116.08,Well,Top Case,7,6,S07,0,0,10,13.924173,50.965596,2022-01-17 12:46:40,1642423600
7,12,G17,115.72,Well,Top Case,9,7,S08,0,1,12,13.923634,50.965507,2022-01-17 12:46:40,1642423600
8,14,G19,115.54,Well,Top Case,11,8,S09,0,1,14,13.923159,50.965430,2022-01-17 12:46:40,1642423600
9,15,G21,115.56,Well,Top Case,12,9,S10,0,0,15,13.922631,50.965332,2022-01-17 12:46:40,1642423600


In [18]:
Get.Isolines(2022, 12, 20, 20 )
df = Get.Isolines_df.copy ()
df

,MonitoringPointID,MonitoringPointName,DiverName,Time,Type,CaseTop,DiverDepth,Value,E,N
0,0,G01,S01,2022-12-20 20:00:00,Rh,119.49,NaN,121.300,13.922657,50.965827
3,23,GWM02,I-2,2022-12-20 20:00:00,h_level,117.98,12.3,NaN,13.923774,50.965905
10,24,GWM03,I-3,2022-12-20 20:00:00,h_level,117.88,12.3,NaN,13.923764,50.965858
17,26,GWM05,I-5,2022-12-20 20:00:00,h_level,117.87,12.3,NaN,13.923778,50.965812
24,15,G24,124,2022-12-20 20:00:00,h_level,119.56,10.8,109.750,13.924817,50.966529
29,20,G29,125,2022-12-20 20:00:00,h_level,120.14,10.8,109.751,13.922332,50.966438
34,21,G30,126,2022-12-20 20:00:00,h_level,118.89,10.8,109.722,13.924894,50.966210
39,18,G27,127,2022-12-20 20:00:00,h_level,118.33,10.8,109.734,13.924870,50.965741


This dataframes are stored within each class as a class attribute. You can retrieve it as follows:

In [35]:
HydroProfile_df = Get.HydroProfile_df
Isolines_df = Get.Isolines_df

and so on...

Just add the suffix **'_df'** to the method name

### 2.3b - Checking up conversion

$$ Value(head) = CaseTop - DiverDepth + DiverReading(Inowas) $$

In [52]:
Get.Isolines(2022, 12, 12, 12 )
df = Get.Isolines_df.copy ()
df ['DiverReading'] = np.round(df.Value - df.CaseTop + df.DiverDepth, 2)
df

,MonitoringPointID,MonitoringPointName,DiverName,Time,Type,CaseTop,DiverDepth,Value,E,N,DiverReading
2,23,GWM02,I-2,2022-12-12 12:00:00,h_level,117.98,12.3,105.680,13.923774,50.965905,0.00
9,24,GWM03,I-3,2022-12-12 12:00:00,h_level,117.88,12.3,105.580,13.923764,50.965858,0.00
16,26,GWM05,I-5,2022-12-12 12:00:00,h_level,117.87,12.3,105.570,13.923778,50.965812,0.00
23,15,G24,124,2022-12-12 12:00:00,h_level,119.56,10.8,109.741,13.924817,50.966529,0.98
28,21,G30,126,2022-12-12 12:00:00,h_level,118.89,10.8,109.693,13.924894,50.966210,1.60
33,20,G29,125,2022-12-12 12:00:00,h_level,120.14,10.8,109.742,13.922332,50.966438,0.40


## 2.3.2 Delete null values from database
### delete where 
$$df.Value - df.CaseTop + df.DiverDepth = 0$$

In [75]:
query = '''
SELECT 
	PointsMeasurements.ID, PointsMeasurements.MonitoringPointID, PointsMeasurements.TimeStamp, PointsMeasurements.VariableID, PointsMeasurements.Value,
	MonitoringPoints.ID, MonitoringPoints.Name as MonitoringPointName, MonitoringPoints.PointID, MonitoringPoints.ReferenceAltitude as CaseTop, 
	Points.ID, Points.E, Points.N, 
	Variables.ID, Variables.Name AS Type,
	WellDiver.DiverID, WellDiver.DiverDepth, WellDiver.MonitoringPointID AS MonitoringPointID2,
	Divers.ID, Divers.Name as DiverName
FROM 
	PointsMeasurements
JOIN
	MonitoringPoints ON PointsMeasurements.MonitoringPointID = MonitoringPoints.ID
JOIN
	Points ON MonitoringPoints.PointID = Points.ID	
JOIN	
	Variables ON PointsMeasurements.VariableID = Variables.ID
JOIN
	WellDiver ON PointsMeasurements.MonitoringPointID = MonitoringPointID2
JOIN
	Divers ON WellDiver.DiverID = Divers.ID
'''

In [76]:
df = pd.read_sql(query, con = Get.connection)

In [77]:
df_ = df.copy()

In [78]:
df_.Type.unique()

array(['t', 'h_level', 'ec_25', 'ec', 'v_batt', 'ph', 'ldo'], dtype=object)

In [ ]:
df = df_ [ df_.Type.isin (['h_level', ''])]

In [73]:
df ['DiverReading'] = np.round(df.Value - df.CaseTop + df.DiverDepth, 2)

In [74]:
df

,ID,MonitoringPointID,TimeStamp,VariableID,Value,ID,MonitoringPointName,PointID,CaseTop,ID,E,N,ID,Type,DiverID,DiverDepth,MonitoringPointID2,ID,DiverName,DiverReading
0,41880,0,1554780600,0,110.389,0,G01,0,119.49,0,13.922657,50.965827,0,h_level,0,NaN,0,0,S01,NaN
1,41881,0,1554784200,0,110.386,0,G01,0,119.49,0,13.922657,50.965827,0,h_level,0,NaN,0,0,S01,NaN
2,41882,0,1554787800,0,110.384,0,G01,0,119.49,0,13.922657,50.965827,0,h_level,0,NaN,0,0,S01,NaN
3,41883,0,1554791400,0,110.381,0,G01,0,119.49,0,13.922657,50.965827,0,h_level,0,NaN,0,0,S01,NaN
4,41884,0,1554795000,0,110.378,0,G01,0,119.49,0,13.922657,50.965827,0,h_level,0,NaN,0,0,S01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562859,1114725,15,1671091200,0,109.863,15,G24,18,119.56,18,13.924817,50.966529,0,h_level,16,10.8,15,16,124,1.1
562860,1114726,15,1671094800,0,109.862,15,G24,18,119.56,18,13.924817,50.966529,0,h_level,16,10.8,15,16,124,1.1
562861,1114727,15,1671098400,0,109.861,15,G24,18,119.56,18,13.924817,50.966529,0,h_level,16,10.8,15,16,124,1.1
562862,1114728,15,1671102000,0,109.859,15,G24,18,119.56,18,13.924817,50.966529,0,h_level,16,10.8,15,16,124,1.1


## 2.4 Saving to CSV

In [31]:
os.chdir(path)

outputsname_dict = {'MonitoringPointData.csv' : MonitoringPointData_df, 'GageData.csv' : GageData_df,
                    'DiverData.csv' : DiverData_df, 'DiverStatus.csv' : DiverStatus_df,
                    'LongTimeSeries.csv' : Get.LongTimeSeries_df, 'ShortTimeSeries.csv' : Get.ShortTimeSeries_df,
                    'Isolines.csv' : Get.Isolines_df, 'HydroProfile.csv' : Get.HydroProfile_df}

def SaveDataFrames():
    folder = 'Data'
    os.chdir(path+f'\\{folder}')
    for i,j in outputsname_dict.items():
        j.to_csv(i, index = False)
        print(f'{i} saved in the {folder} folder')

if you wish to save it, uncomment and run the line below

In [32]:
SaveDataFrames()

MonitoringPointData.csv saved in the Data folder
GageData.csv saved in the Data folder
DiverData.csv saved in the Data folder
DiverStatus.csv saved in the Data folder
LongTimeSeries.csv saved in the Data folder
ShortTimeSeries.csv saved in the Data folder
Isolines.csv saved in the Data folder
HydroProfile.csv saved in the Data folder
